# Library import

In [1]:
import tensorflow as tf
import keras.api as keras
from keras.api import layers
from keras.api.layers import Conv2D, LeakyReLU, Flatten, Dropout, Dense
import matplotlib.pyplot as plt
import numpy as np

2025-03-16 15:08:14.189237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742148494.204726  199487 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742148494.209342  199487 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-16 15:08:14.224925: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Functions

In [3]:
def normalize(image):
    image = tf.cast(image, tf.float32) / 255.0
    image = (image - 0.5) * 2
    return image

# Parameters

In [10]:
image_size = (128, 128)
channel = 1
BUFFER_SIZE = 60000
BATCH_SIZE = 64
dataset_path = "../../data/train/"

# Dataset load

In [11]:
train_ds = keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    label_mode=None,
    color_mode='grayscale',
    image_size=image_size,
    batch_size=BATCH_SIZE
)

Found 2422 files.


I0000 00:00:1742148807.063244  199487 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9910 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6
